## Question 10

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.analysis_helpers import side_by_side_bar_plot, get_value_counts, distribution_plots
from scipy.stats import mannwhitneyu, kruskal, rankdata

In [4]:
def normalized_mean_rank(series, method="average"):
    """
    """
    rank_data = rankdata(series, method=method)
    max_rank_data = np.max(rank_data)
    return np.mean(rank_data) / max_rank_data
    

In [5]:
def franchise_analysis(df, franchise_name):
    """
    """

    # empty lists to hold movie names and movie data
    franchise_movies = []
    franchise_survey_data = []
    franchise_mean_normalized_ranks = []
    franchise_medians = []
    franchise_means = []

    # get all movies of same franchise
    for col in df.columns:
        if franchise_name in col:
            movie_data = df[col].copy().dropna()  # get data for movie
            franchise_movies.append(col)  # add column name to list
            franchise_survey_data.append(movie_data)  # add movie data to list
            franchise_mean_normalized_ranks.append(normalized_mean_rank(movie_data))
            franchise_medians.append(movie_data.median())
            franchise_means.append(movie_data.mean())
    
    # run test
    H, p = kruskal(*franchise_survey_data)

    # collate results into singular dictionary to return
    results = {
        "franchise": franchise_name,
        "movies": franchise_movies,
        "mean_normalized_ranks": franchise_mean_normalized_ranks,
        "median_ratings": franchise_medians,
        "average_ratings": franchise_means,
        "p": p,
        "H": H
    }

    return results

In [6]:
# Read in reviews data csv file
df = pd.read_csv("../data/movieReplicationSet.csv")

# list of franchises
franchises = [
    "Star Wars",
    "Harry Potter",
    "The Matrix",
    "Indiana Jones",
    "Jurassic Park",
    "Pirates of the Caribbean",
    "Toy Story",
    "Batman"
]

# empty lists to hold analysis results
Hvalues = []
pvalues = []

for movie_series in franchises:
    res = franchise_analysis(df, movie_series)
    Hvalues.append(res["H"])
    pvalues.append(res["p"])
    
    print(f"Franchise: {movie_series} \t # of Movies: {len(res['movies'])}")

df_results = pd.DataFrame({
    "Franchise": franchises,
    "p": pvalues,
    "H": Hvalues,
    #"Movies": movies_per_franchise
})

df_results["significance"] = df_results["p"] < 0.005

Franchise: Star Wars 	 # of Movies: 6
Franchise: Harry Potter 	 # of Movies: 4
Franchise: The Matrix 	 # of Movies: 3
Franchise: Indiana Jones 	 # of Movies: 4
Franchise: Jurassic Park 	 # of Movies: 3
Franchise: Pirates of the Caribbean 	 # of Movies: 3
Franchise: Toy Story 	 # of Movies: 3
Franchise: Batman 	 # of Movies: 3


In [7]:
df_results

,Franchise,p,H,significance
0,Star Wars,8.016477e-48,230.584175,True
1,Harry Potter,3.433195e-01,3.331231,False
2,The Matrix,3.123652e-11,48.378867,True
3,Indiana Jones,6.272776e-10,45.794163,True
4,Jurassic Park,7.636930e-11,46.590881,True
5,Pirates of the Caribbean,3.290129e-05,20.643998,True
6,Toy Story,5.065805e-06,24.385995,True
7,Batman,4.225297e-42,190.534969,True
